## Chat Models - Tagging Documents

In [1]:
%pip install langchain langchain_openai nest_asyncio lxml beautifulsoup4 langchain-community --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 44.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 66.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 399.9/399.9 kB 27.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 292.2/292.2 kB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.0/383.0 kB 28.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 52.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 20.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 k

In [2]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass()

··········


In [3]:
# fixes a bug with asyncio and jupyter
import nest_asyncio

nest_asyncio.apply()

In [4]:
from langchain.document_loaders.sitemap import SitemapLoader
from langchain_openai.chat_models import ChatOpenAI
from langchain.chains import create_tagging_chain, create_tagging_chain_pydantic
import pandas as pd

In [5]:
sitemap_loader = SitemapLoader(web_path="https://understandingdata.com/sitemap.xml")
sitemap_loader.requests_per_second = 5
docs = sitemap_loader.load()

Fetching pages: 100%|##########| 101/101 [00:03<00:00, 25.71it/s]


In [8]:
# Schema
schema = {
    "properties": {
        "sentiment": {"type": "string" },
        "aggressiveness": {"type": "integer"},
        "primary_topic": {"type": "string", "description": "The main topic of the document."},
    },
     "required": ["primary_topic", "sentiment", "aggressiveness"],
}

# LLM
llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo")
chain = create_tagging_chain(schema, llm, output_key='output')

In [9]:
results = []

# Remove the 0:10 to run on all documents:
for index, doc in enumerate(docs[0:10]):
    print(f"Processing doc {index +1}")
    chain_result = chain.invoke({'input': doc.page_content})
    results.append(chain_result['output'])

Processing doc 1
Processing doc 2
Processing doc 3
Processing doc 4
Processing doc 5
Processing doc 6
Processing doc 7
Processing doc 8
Processing doc 9
Processing doc 10


In [10]:
results

[{'sentiment': 'positive',
  'aggressiveness': 3,
  'primary_topic': 'Software & Data Engineering'},
 {'sentiment': 'positive', 'aggressiveness': 0, 'primary_topic': 'technology'},
 {'sentiment': 'positive', 'aggressiveness': 0, 'primary_topic': 'Contact'},
 {'sentiment': 'positive',
  'aggressiveness': 2,
  'primary_topic': 'Software & Data Engineering Services'},
 {'sentiment': 'positive',
  'aggressiveness': 0,
  'primary_topic': 'Software & Data Engineering'},
 {'sentiment': 'neutral',
  'aggressiveness': 0,
  'primary_topic': 'Data Engineering'},
 {'sentiment': 'positive',
  'aggressiveness': 0,
  'primary_topic': 'Data Engineering Services'},
 {'sentiment': 'positive',
  'aggressiveness': 3,
  'primary_topic': 'React Software Development'},
 {'sentiment': 'positive',
  'aggressiveness': 0,
  'primary_topic': 'Python software development'},
 {'sentiment': 'positive',
  'aggressiveness': 2,
  'primary_topic': 'Python software development'}]

In [11]:
df = pd.DataFrame(results)

In [12]:
df

,sentiment,aggressiveness,primary_topic
0,positive,3,Software & Data Engineering
1,positive,0,technology
2,positive,0,Contact
3,positive,2,Software & Data Engineering Services
4,positive,0,Software & Data Engineering
5,neutral,0,Data Engineering
6,positive,0,Data Engineering Services
7,positive,3,React Software Development
8,positive,0,Python software development
9,positive,2,Python software development


In [13]:
docs[0].metadata

{'source': 'https://understandingdata.com/',
 'loc': 'https://understandingdata.com/',
 'lastmod': '2024-09-29T20:35:28.914Z',
 'changefreq': 'monthly',
 'priority': '1.0'}

In [14]:
# Combine the URLs with the results
df['url'] = [doc.metadata['source'] for doc in docs[0:10]]


In [ ]:
df

,sentiment,aggressiveness,primary_topic,url
0,positive,0.5,software engineering,https://understandingdata.com/
1,passionate,0.0,innovative solutions,https://understandingdata.com/about/
2,positive,0.5,contact,https://understandingdata.com/contact/
3,positive,0.5,software & data engineering services,https://understandingdata.com/services/
4,positive,0.5,software and data engineering,https://understandingdata.com/projects/
5,positive,0.5,data engineering,https://understandingdata.com/posts/
6,positive,0.5,data engineering services,https://understandingdata.com/services/data-en...
7,positive,0.5,React Software Development,https://understandingdata.com/services/react-s...
8,positive,0.5,Python software development,https://understandingdata.com/services/python-...
9,positive,0.5,Python software development,https://understandingdata.com/services/python-...
